In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
from sklearn.metrics import mean_squared_error

In [2]:
data=pd.read_csv('./ratings.csv')

In [3]:
print(data.head)
print(data.shape)

<bound method NDFrame.head of          userId  movieId  rating   timestamp
0             1        2     3.5  1112486027
1             1       29     3.5  1112484676
2             1       32     3.5  1112484819
3             1       47     3.5  1112484727
4             1       50     3.5  1112484580
...         ...      ...     ...         ...
1048570    7120      168     5.0  1175543061
1048571    7120      253     4.0  1175542225
1048572    7120      260     5.0  1175542035
1048573    7120      261     4.0  1175543376
1048574    7120      266     3.5  1175542454

[1048575 rows x 4 columns]>
(1048575, 4)


In [4]:
sparse_features = ["userId", "movieId"]
target = ['rating']
data[sparse_features] = data[sparse_features].fillna('-1', )
sparse_feature_columns = [SparseFeat(feat, data[feat].nunique())
                        for feat in sparse_features]
print(sparse_feature_columns)

[SparseFeat:userId, SparseFeat:movieId]


In [5]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [6]:
dnn_feature_columns = sparse_feature_columns 
linear_feature_columns = sparse_feature_columns 

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(feature_names)

['userId', 'movieId']


In [7]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}


使用Wide & Deep进行训练

In [8]:
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values,batch_size=256, epochs=10, verbose=True, validation_split=0.2,)

W1111 13:41:34.698973 20736 deprecation.py:506] From c:\users\lx\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1111 13:41:35.214530 20736 deprecation.py:506] From c:\users\lx\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\autograph\impl\api.py:253: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W1111 13:41:35.254910 20736 deprecation.py:506] From c:\users\lx\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow

Train on 671088 samples, validate on 167772 samples
Epoch 1/10
671088/671088 [==============================] - 41s 61us/sample - loss: 0.9379 - mean_squared_error: 0.9365 - val_loss: 0.7357 - val_mean_squared_error: 0.7331
Epoch 2/10
671088/671088 [==============================] - 44s 66us/sample - loss: 0.7161 - mean_squared_error: 0.7123 - val_loss: 0.7198 - val_mean_squared_error: 0.7147
Epoch 3/10
671088/671088 [==============================] - 46s 69us/sample - loss: 0.6912 - mean_squared_error: 0.6849 - val_loss: 0.7146 - val_mean_squared_error: 0.7072
Epoch 4/10
671088/671088 [==============================] - 38s 56us/sample - loss: 0.6713 - mean_squared_error: 0.6627 - val_loss: 0.7092 - val_mean_squared_error: 0.6997
Epoch 5/10
671088/671088 [==============================] - 41s 61us/sample - loss: 0.6571 - mean_squared_error: 0.6465 - val_loss: 0.7228 - val_mean_squared_error: 0.7113
Epoch 6/10
671088/671088 [==============================] - 42s 63us/sample - loss: 0.64

使用Wide & Deep进行预测

In [9]:
pred_ans = model.predict(test_model_input, batch_size=256)

mse=round(mean_squared_error(test[target].values,pred_ans),4)
rmse=mse**0.5
print('test RMSE',rmse)

test RMSE 0.848528137423857
